# Code source du chapitre 1
*mis à jour le 01.07.2022*

In [8]:
import geopandas as gpd
import os

PATH = os.path.join("_DATA","volumesbatisparis_ensapvs.geojson")

# Lecture du fichier des volumes bâtis de l'ENSAPVS
data = gpd.read_file(PATH)

data.head(4)

,objectid,n_sq_pf,d_maj,l_src,l_b_u,l_plan_h,h_et_max,b_rdc,n_ar,m2_pl_tot,...,shape_area,c_src,nb_pl,n_sq_vb,shape_len,c_nat_b,n_sq_ar,y,x,geometry
0,536092,750031416.0,2010-07-21 04:00:00+02:00,Fiche parcellaire et terrain certifié,None,R+3,3.0,1.0,13,281.214948,...,0.0,T,4.0,750169268,0.0,V,750000013,125224.417845,603527.957994,"POLYGON ((2.38462 48.82753, 2.38463 48.82753, ..."
1,536130,750031416.0,2010-07-21 04:00:00+02:00,Fiche parcellaire et terrain certifié,5a8,None,8.0,0.0,13,9.842935,...,0.0,T,4.0,750169701,0.0,U,750000013,125193.058830,603540.498013,"POLYGON ((2.38475 48.82722, 2.38476 48.82721, ..."
2,536134,750031416.0,2010-07-21 04:00:00+02:00,Fiche parcellaire et terrain certifié,Ra1_et_3a9,None,9.0,1.0,13,52.982843,...,0.0,T,9.0,750169705,0.0,U,750000013,125203.050906,603553.631901,"POLYGON ((2.38493 48.82732, 2.38495 48.82730, ..."
3,536186,750031406.0,2010-07-21 04:00:00+02:00,Fiche parcellaire et terrain certifié,2a8,None,8.0,0.0,13,229.771164,...,0.0,T,7.0,750170552,0.0,U,750000013,125123.348550,603439.557492,"POLYGON ((2.38354 48.82665, 2.38323 48.82652, ..."


In [9]:
# Sélection et renommage des colonnes des colonnes utiles
data = data[["geometry","m2_pl_tot","h_et_max","l_b_u"]]
data = data.rename(columns={
    "m2_pl_tot" : "surface",
    "h_et_max" : "hauteur",
    "l_b_u" : "hauteur_paf"
    })

data.head(4)

,geometry,surface,hauteur,hauteur_paf
0,"POLYGON ((2.38462 48.82753, 2.38463 48.82753, ...",281.214948,3.0,None
1,"POLYGON ((2.38475 48.82722, 2.38476 48.82721, ...",9.842935,8.0,5a8
2,"POLYGON ((2.38493 48.82732, 2.38495 48.82730, ...",52.982843,9.0,Ra1_et_3a9
3,"POLYGON ((2.38354 48.82665, 2.38323 48.82652, ...",229.771164,8.0,2a8


In [10]:
# Aperçu des différentes notations de porte-à-faux
notations_paf = {}
for txt in data["hauteur_paf"].dropna():
    notations_paf[len(txt)] = txt

notations_paf

{3: '2a4',
 10: '2a4_et_7a8',
 12: 'encorbt_au_5',
 19: 'Ra1_et_encorbt_au_5',
 8: 'R_et_3a9',
 9: 'Auvent_n1',
 4: '4a11',
 17: 'R_et_encorbt_au_3',
 15: 'Ra1_et_3_et_5a8'}

In [11]:
# Conversion de la notation textuelle des hauteurs de bâtiments et des porte-à-faux en intervalles numériques
def texte_to_num(volume, h_etage=3):
            
    if volume["hauteur_paf"]:
        # Liste vide pour contenir les notations numériques
        output = []
        # Séparation des sous-notations au niveau du "_et_"
        # Si absent, la notation sera conservée telle quelle
        intervalles = volume["hauteur_paf"].split("_et_")
        # Itération à travers les sous-notations
        for interv in intervalles:
            # filtrage par syntaxe
            if "encorbt_au_" in interv:
                n = int(interv.strip("encorbt_au_"))
                if n == volume["hauteur"]:
                    output.append([n*h_etage,(n+1)*h_etage])
                else:
                    output.append([(n+1)*h_etage,(volume["hauteur"]+1)*h_etage])
            elif "auvent_n" in interv:
                n = int(interv.strip("auvent_n"))
                output.append([n*h_etage,(n+1)*h_etage])
            elif "a" in interv and "_avec_equiv_" not in interv:
                if "R" in interv:
                    output.append([0,(int(interv.strip("Ra"))+1)*h_etage])
                else:
                    output.append([(int(h)+1)*h_etage for h in interv.split("a")])
            elif interv == "R":
                output.append([0,h_etage])
            elif "_avec_equiv_" in interv:
                n = int(interv[-2])
                if interv[-2] == 0:
                    n = int(interv[-3:-2])
                output.append([0,h_etage*n])
        # Remplacement par la notation numérique
        volume["hauteur_paf"] = output

    volume["hauteur"] = (volume["hauteur"] + 1)*h_etage
    return volume

data = data.apply(texte_to_num,axis=1)
data

,geometry,surface,hauteur,hauteur_paf
0,"POLYGON ((2.38462 48.82753, 2.38463 48.82753, ...",281.214948,12.0,None
1,"POLYGON ((2.38475 48.82722, 2.38476 48.82721, ...",9.842935,27.0,"[[18, 27]]"
2,"POLYGON ((2.38493 48.82732, 2.38495 48.82730, ...",52.982843,30.0,"[[0, 6], [12, 30]]"
3,"POLYGON ((2.38354 48.82665, 2.38323 48.82652, ...",229.771164,27.0,"[[9, 27]]"
4,"POLYGON ((2.38304 48.82670, 2.38315 48.82659, ...",310.513295,30.0,"[[12, 30]]"
...,...,...,...,...
330,"POLYGON ((2.38469 48.82720, 2.38469 48.82720, ...",1.632114,15.0,"[[9, 15]]"
331,"POLYGON ((2.38451 48.82752, 2.38452 48.82751, ...",3.082522,12.0,"[[0, 3], [9, 12]]"
332,"POLYGON ((2.38453 48.82753, 2.38454 48.82752, ...",5.338095,18.0,"[[0, 6], [15, 18]]"
333,"POLYGON ((2.38463 48.82775, 2.38464 48.82774, ...",4.296431,18.0,"[[15, 18]]"


In [14]:
%%writefile chapitre_1_dump.py
# Export pour réutilisation dans le code du chapitre 2
import geopandas as gpd

def texte_to_num(volume, h_etage=3):
            
    if volume["hauteur_paf"]:
        # Liste vide pour contenir les notations numériques
        output = []
        # Séparation des sous-notations au niveau du "_et_"
        # Si absent, la notation sera conservée telle quelle
        intervalles = volume["hauteur_paf"].split("_et_")
        # Itération à travers les sous-notations
        for interv in intervalles:
            # filtrage par syntaxe
            if "encorbt_au_" in interv:
                n = int(interv.strip("encorbt_au_"))
                if n == volume["hauteur"]:
                    output.append([n*h_etage,(n+1)*h_etage])
                else:
                    output.append([(n+1)*h_etage,(volume["hauteur"]+1)*h_etage])
            elif "auvent_n" in interv:
                n = int(interv.strip("auvent_n"))
                output.append([n*h_etage,(n+1)*h_etage])
            elif "a" in interv and "_avec_equiv_" not in interv:
                if "R" in interv:
                    output.append([0,(int(interv.strip("Ra"))+1)*h_etage])
                else:
                    output.append([(int(h)+1)*h_etage for h in interv.split("a")])
            elif interv == "R":
                output.append([0,h_etage])
            elif "_avec_equiv_" in interv:
                n = int(interv[-2])
                if interv[-2] == 0:
                    n = int(interv[-3:-2])
                output.append([0,h_etage*n])
        # Remplacement par la notation numérique
        volume["hauteur_paf"] = output

    volume["hauteur"] = (volume["hauteur"] + 1)*h_etage
    return volume

def import_data_batiments(path, h_etage):

    data = gpd.read_file(path)
    data = data[["geometry","m2_pl_tot","h_et_max","l_b_u"]]
    data = data.rename(columns={
        "m2_pl_tot" : "surface",
        "h_et_max" : "hauteur",
        "l_b_u" : "hauteur_paf"
        })

    data = data.apply(texte_to_num,axis=1)

    return data

Writing chapitre1.py
